In [2]:
import pandas as pd
import numpy as np
import EvaluateModels as em
import boto3

## defining bucket
s3 = boto3.resource('s3')

bucket_name = 'gabriel-de-medeiros'

bucket = s3.Bucket(bucket_name)

## specifying file from the bucket
file_key = 'CleanTrainData.csv'
file_key2 = 'CleanTestData.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
train_data = pd.read_csv(file_content_stream, index_col = 0)
test_data = pd.read_csv(file_content_stream2, index_col = 0)

train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)

train_data = train_data.select_dtypes(exclude=['object'])
test_data = test_data.select_dtypes(exclude=['object'])

X_t = train_data.dropna()
X_tst = test_data.dropna()


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [20]:

train_data = X_t.iloc[0:50000,:]
test_data = X_tst.iloc[0:50000,:]


X_train, Y_train = train_data.drop(columns = 'is_fraud'), train_data['is_fraud']


X_train.columns

Index(['uses_per_category', 'std_dist', 'std_dist_by_category',
       'std_amt_by_category', 'std_amt', 'avg_amt', 'avg_by_category',
       'total_uses', 'hour_of_the_day', 'uses_per_hour', 'month',
       'uses_per_month', 'uses_per_day', 'avg_distance',
       'avg_distance_by_category', 'amt', 'zip', 'lat', 'long', 'city_pop',
       'unix_time', 'merch_lat', 'merch_long', 'distance', 'age',
       'diff_by_card_trans', 'diff_first_last', 'purchase_>_avg',
       'purchase_>_avg_by_category', 'purchase_>_distance',
       'purchase_>_distance_by_category', 'amt_>_1_std', 'amt_>_2_std',
       'amt_>_1_std_by_cagegory', 'amt_>_2_std_by_cagegory', 'dist_>_1_std',
       'dist_>_2_std', 'dist_>_1_std_by_cagegory', 'dist_>_2_std_by_cagegory'],
      dtype='object')

In [13]:
X = X_train
Y = Y_train

import Get_Variable_Importance as gvi

print('DecisionTrees')
decision_trees_variable_importance = gvi.Decision_Tree_Importance(X,Y)

print('RandomForest')
random_forest_variable_importance = gvi.Random_Forest_Importance(X,Y)

print('ImportantVariables')
df_importance_columns = gvi.Getting_Best_Model(decision_trees_variable_importance,random_forest_variable_importance,X,Y)

DecisionTrees


100%|██████████| 80/80 [00:07<00:00, 10.48it/s]


RandomForest


100%|██████████| 800/800 [01:18<00:00, 10.26it/s]

ImportantVariables


In [21]:
X_test = test_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_test = test_data['is_fraud']

X_train = train_data[df_importance_columns.drop(columns = 'is_fraud').columns]
Y_train = train_data['is_fraud']

In [22]:
## Decision Trees
DTC_results = em.DecisionTreesResults(X_test, X_train, Y_test, Y_train)
DTC_best_model = em.DecisionTreesBestModel(DTC_results)

DecisionTrees


100%|██████████| 8/8 [00:04<00:00,  1.95it/s]


In [23]:
DTC_best_model

,max_depth,cut_off,accuracy,recall,Performance
10,4,0.10,0.9931,0.756198,0.858608
11,4,0.15,0.9931,0.756198,0.858608


In [24]:
LR_results = em.LogisticRegressionResults(X_test, X_train, Y_test, Y_train)

LogisticRegression


100%|██████████| 10/10 [00:00<00:00, 21.16it/s]


In [25]:
LR_results

,cut_off,accuracy,recall,Performance
0,0.10,0.99516,0.0,0.0
1,0.15,0.99516,0.0,0.0
2,0.20,0.99516,0.0,0.0
3,0.25,0.99516,0.0,0.0
4,0.30,0.99516,0.0,0.0
5,0.35,0.99516,0.0,0.0
6,0.40,0.99516,0.0,0.0
7,0.45,0.99516,0.0,0.0
8,0.50,0.99516,0.0,0.0
9,0.55,0.99516,0.0,0.0


In [ ]:
NN_results = em.NeuralNetworksResults(X_test, X_train, Y_test, Y_train)

In [ ]:
RF_results = em.RandomForestResults(X_test, X_train, Y_test, Y_train)
RF_best_model = em.DecisionTreesBestModel(RF_results)

In [ ]:
# ## Random Forest
# RF_results = em.RandomForestResults(X_test, X_train, Y_test, Y_train)
# RF_best_model = em.DecisionTreesBestModel(RF_results)

## Neural Networks
#NN_results = em.NeuralNetworksResults(X_test, X_train, Y_test, Y_train)

# ## SVC
# SVC_results = em.SupportVectorMachineResults(X_test, X_train, Y_test, Y_train)
# SVC_best_model = em.SvcBestModel(SVC_results)

# ## Logistic Regression
# LR_results = em.LogisticRegressionResults(X_test, X_train, Y_test, Y_train)

# ## AdaBoost Decision Trees
# ADA_DTC = em.AdaBoostDecisionTreesResults(X_test, X_train, Y_test, Y_train, DTC_best_model)

# ## AdaBoost SVC
# ADA_SVC= em.AdaBoostSvmResults(X_test, X_train, Y_test, Y_train, SVC_best_model)

## GradientBoosting
# GBC_results = em.GradientBoostingResults(X_test, X_train, Y_test, Y_train)



# DTC_results.to_csv('CreditCardsFraudDetection/Algorithm/DTC_results.csv', index = False)
# DTC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/DTC_best_model.csv', index = False)
# RF_results.to_csv('CreditCardsFraudDetection/Algorithm/RF_results.csv', index = False)
# RF_best_model.to_csv('CreditCardsFraudDetection/Algorithm/RF_best_model.csv', index = False)
# NN_results.to_csv('CreditCardsFraudDetection/Algorithm/NN_results.csv', index = False)
# SVC_results.to_csv('CreditCardsFraudDetection/Algorithm/SVC_results.csv', index = False)
# SVC_best_model.to_csv('CreditCardsFraudDetection/Algorithm/SVC_best_model.csv', index = False)
# LR_results.to_csv('CreditCardsFraudDetection/Algorithm/LR_results.csv', index = False)
# ADA_DTC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_DTC.csv', index = False)
# ADA_SVC.to_csv('CreditCardsFraudDetection/Algorithm/ADA_SVC.csv', index = False)
# GBC_results.to_csv('CreditCardsFraudDetection/Algorithm/GBC_results.csv', index = False)